In [0]:
#Do some additional work on look at the input data and understanding the layers / embeddings / vectorizing, ect...
#Dig more into the weights and neurons and try to see vanishing and exploding gradients
#Any other regularizaionts that can be applied
#using windowxs in RNN to shorten length of layers
#Stateful RNNs
#Combining RNNs and Conv1d
#Do some more exploration with the raw data

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import h5py
import os
import time
from random import sample
import datetime
from collections import Counter
import sys
from tqdm import tqdm, notebook
from tensorflow.keras.datasets import reuters
pd.options.display.max_colwidth = 100

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import sparse_categorical_crossentropy
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import ParameterGrid

word_index = reuters.get_word_index()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


557056/550378 [==============================] - 0s 0us/step


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_dir = '/content/gdrive/My Drive/'
base_dir = root_dir + 'Northwestern/Artificial Intelligence and Deep Learning/Assignment 3/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:


voc_size = 10000

(train_data_raw, train_labels_raw), (test_data_raw, test_labels_raw) = reuters.load_data(num_words=voc_size)
word_index = reuters.get_word_index()
train_data_raw.shape, test_data_raw.shape

((8982,), (2246,))

In [0]:
def import_data(voc_size):

    (train_data_raw, train_labels_raw), (test_data_raw, test_labels_raw) = reuters.load_data(num_words=voc_size)
    # word_index = reuters.get_word_index()

    return train_data_raw, train_labels_raw, test_data_raw, test_labels_raw

In [0]:
def top_label_data(X, y):

    topics_tpl, topics_freq = zip(*Counter(y).most_common(9))
    X_top, y_top = zip(*((x_samp, y_samp) for x_samp, y_samp in zip(X, y) if y_samp in topics_tpl))
    X_top, y_top = np.array(X_top), np.array(y_top)

    keys = Counter(y_top).keys()
    conv = dict(zip(sorted(keys), range(0,9)))
    y_top = [conv[l] for l in y_top]

    return X_top, y_top

In [0]:
def max_len(train_data, train_labels, percentile, top):

    if top == False:
        pass

    elif top == True:
        train_data, train_labels = top_label_data(train_data, train_labels)

    train_data_len = [len(w) for w in train_data]
    maxlen = int(np.percentile(train_data_len, percentile))

    return maxlen

In [0]:
def return_max_lens(train_data, train_labels, top):

    max_lengths = []
    for percentile in np.arange(20, 120, 20):
        ml = max_len(train_data, train_labels, percentile, top = top)
        max_lengths.append(ml)

    return max_lengths

In [0]:
def prepare_data(train_data, train_labels, test_data, test_labels, max_len,
                 top, embedding = False):

    if top == False:
        pass

    elif top == True:
        train_data, train_labels = top_label_data(train_data, train_labels)
        test_data, test_labels = top_label_data(test_data, test_labels)

    if embedding == False:
        train_data = vectorize_sequences(train_data)
        test_data = vectorize_sequences(test_data)
        # maxlen = None
        
    elif embedding == True:
        # maxlen = max_len(train_data, train_labels, top)
        train_data = preprocessing.sequence.pad_sequences(train_data, maxlen=max_len)
        test_data = preprocessing.sequence.pad_sequences(test_data, maxlen=max_len)

    train_data, val_data, train_labels, val_labels = train_test_split(
        train_data, train_labels, test_size = 0.15)
    train_labels = to_categorical(train_labels)
    val_labels = to_categorical(val_labels)
    test_labels = to_categorical(test_labels)

    return train_data, train_labels, val_data, val_labels, test_data, test_labels

In [0]:
def tensor_slices(train_data, train_labels, val_data, val_labels, test_data, test_labels):

    train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(100).batch(100)
    val_ds = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(100)
    test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

    return train_ds, val_ds, test_ds

In [0]:
def embedding_matrix(embedding_file, voc_size):

    embeddings_index = {}
    f = open(os.path.join(base_dir, embedding_file))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype = 'float32')
        embeddings_index[word] = coefs
    f.close()

    embedding_dim = int(embedding_file.split('.')[-2].replace('d', ''))

    embedding_matrix = np.zeros((voc_size, embedding_dim))
    for word, i in word_index.items():
        if i < voc_size:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

    return embedding_matrix, embedding_dim

In [0]:
def build_embed_model(output_dim, max_len, voc_size, top):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()
    model.add(Embedding(input_dim=voc_size, output_dim = output_dim, input_length = max_len))
    model.add(Flatten())
    model.add(Dense(units = topics, activation = 'softmax'))

    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_rnn_1(top, max_len, voc_size, output_dim, l1_units, 
                pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(SimpleRNN(units = l1_units, activation = 'tanh'))
    model.add(Dense(units = topics, activation = 'softmax'))

    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_rnn_2(top, max_len, voc_size, output_dim, l1_units, l2_units,
                pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(SimpleRNN(units = l1_units, activation = 'tanh', return_sequences = True))
    model.add(SimpleRNN(units = l2_units, activation = 'tanh'))
    model.add(Dense(units = topics, activation = 'softmax'))

    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_gru_1(top, max_len, voc_size, output_dim, l1_units, dropout,
                pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(GRU(units = l1_units, activation = 'tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(units = topics, activation = 'softmax'))

    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_gru_2(top, max_len, voc_size, output_dim, l1_units, l2_units, dropout,
                pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))


    model.add(GRU(units = l1_units, activation = 'tanh', return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = l2_units, activation = 'tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(units = topics, activation = 'softmax'))

    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_lstm_1(top, max_len, voc_size, output_dim, l1_units, dropout,
                pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(LSTM(units = l1_units, activation = 'tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(units = topics, activation = 'softmax'))

    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_lstm_2(top, max_len, voc_size, output_dim, l1_units, l2_units, dropout,
                pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))


    model.add(LSTM(units = l1_units, activation = 'tanh', return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(units = l2_units, activation = 'tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(units = topics, activation = 'softmax'))

    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_dnn_1(top, max_len, voc_size, output_dim, l1_units, l2_units, dropout,
                pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(Flatten())
    model.add(Dense(units = l1_units, activation = 'relu', input_shape = (voc_size,)))
    model.add(Dropout(dropout))
    model.add(Dense(units = l2_units, activation = 'relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units = topics, activation = 'softmax'))


    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_dnn_2(top, max_len, voc_size, output_dim, l1_units, l2_units, l3_units, 
                dropout, pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(Flatten())
    model.add(Dense(units = l1_units, activation = 'relu', input_shape = (voc_size,)))
    model.add(Dense(units = l2_units, activation = 'relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units = l3_units, activation = 'relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units = topics, activation = 'softmax'))


    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_cnn_1(top, max_len, voc_size, output_dim, l1_filters, 
                dropout, pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(Conv1D(filters = l1_filters, kernel_size = 3, activation = 'relu'))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(units = topics, activation = 'softmax'))


    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def build_cnn_2(top, max_len, voc_size, output_dim, l1_filters, l2_filters, 
                dropout, pretrained_embed = False):

    if top == False:
        topics = 46
    
    elif top == True:
        topics = 9

    model = Sequential()

    if pretrained_embed == True:
        embedding_file = 'glove.6B.100d.txt'
        embed_matrix, embed_dim = embedding_matrix(embedding_file, voc_size)
        output_dim = None
        output_dim = embed_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))
        model.layers[0].set_weights([embed_matrix])
        model.layers[0].trainable = False
        output_dim = None
        output_dim = embed_dim
    elif pretrained_embed == False:
        output_dim = output_dim
        model.add(Embedding(input_dim = voc_size, output_dim = output_dim, input_length = max_len))

    model.add(Conv1D(filters = l1_filters, kernel_size = 5, activation = 'relu'))
    model.add(MaxPooling1D(5))
    model.add(Conv1D(filters = l2_filters, kernel_size = 3, activation = 'relu'))    
    model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(units = topics, activation = 'softmax'))


    model.compile(
                    optimizer = 'rmsprop',
                    loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])

    return model

In [0]:
def run_param_grid(models, parameters, embedding):



    model_name = []
    num_parameters = []
    structure = []
    conv_net = []
    accuracy = []
    val_accuracy = []
    loss = []
    test_accuracy = []
    test_loss = []
    val_loss = []
    elapsed_time = []

    for idx, model in enumerate(models):

        
        grid = ParameterGrid(parameters[idx])
        for params in tqdm(grid):

            train_data_raw, train_labels_raw, test_data_raw, test_labels_raw = import_data(params['voc_size'])

            train_data, train_labels, val_data, val_labels, test_data, test_labels = prepare_data(train_data_raw, train_labels_raw, 
                test_data_raw, test_labels_raw, params['max_len'],
                top = params['top'], embedding = embedding)
            
            keras.backend.clear_session()
        
            start = time.time()
            
            clf = KerasClassifier(build_fn=model,  **params)
            history = clf.fit(train_data,
                                train_labels,
                                epochs = 10,
                                batch_size = 128,
                                validation_data = (val_data, val_labels))
            
            end = time.time()
            t_loss, t_acc = history.model.evaluate(test_data, test_labels)

            model_name.append(f'model_{idx + 1}')
            structure.append(params)
            num_parameters.append(history.model.count_params())
            accuracy.append(history.history['accuracy'][-1])
            val_accuracy.append(history.history['val_accuracy'][-1])
            test_accuracy.append(t_acc)
            loss.append(history.history['loss'][-1])
            val_loss.append(history.history['val_loss'][-1])
            test_loss.append(t_loss)            
            elapsed_time.append(end - start)

    model_outputs = {'model':model_name, 
                    #  'conv_net':conv_net, 
                     'structure':structure, 
                     'num_parameters':num_parameters, 
                     'accuracy':accuracy, 
                     'val_accuracy':val_accuracy,
                     'test_accuracy':test_accuracy, 
                     'loss':loss, 
                     'val_loss':val_loss, 
                     'test_loss':test_loss,
                     'elapsed_time':elapsed_time}

    return model_outputs

In [0]:
#Have a larger list to test embeddings specifically and use a narrowed list when testing other things

# models = [build_model_1_grid_search, build_model_2_grid_search,
#           build_model_3_grid_search, build_model_4_grid_search]

models = [build_cnn_2]

parameters = [
    # {'voc_size': [1000, 10000, 50000],
    # 'top': [True, False],
    # 'output_dim':np.array([8, 64, 128, 256, 512]), 
    # 'max_len':np.array([20, 100, 280, 2376])}, 
    # {'voc_size': [10000],
    # 'top': [True, False],
    # 'output_dim':np.array([8, 128, 512]), 
    # 'max_len':np.array([100, 280]),
    # 'l1_units':np.array([16, 32, 64, 128, 256]),
    # 'pretrained_embed':[True, False]},
    # {'voc_size': [10000],
    # 'top': [True, False],
    # 'output_dim':np.array([8, 128, 512]), 
    # 'max_len':np.array([100, 280]),
    # 'l1_units':np.array([16, 64, 256]),
    # 'l2_units':np.array([16, 32, 256]),
    # 'pretrained_embed':[True, False]},
    # {'voc_size': [10000],
    # 'top': [True, False],
    # 'output_dim':np.array([8, 128, 512]), 
    # 'max_len':np.array([100, 280]),
    # 'l1_units':np.array([16, 32, 100, 256]),
    # 'dropout':np.array([0, 0.25]),
    # 'pretrained_embed':[True, False]},
    # {'voc_size': [10000],
    # 'top': [True, False],
    # 'output_dim':np.array([128, 512]), 
    # 'max_len':np.array([100, 280]),
    # 'l1_units':np.array([16, 100, 256]),
    # 'l2_units':np.array([32, 100]),
    # 'dropout':np.array([0, 0.25]),
    # 'pretrained_embed':[True, False]},
    # {'voc_size': [10000],
    # 'top': [True, False],
    # 'output_dim':np.array([8, 128, 512]), 
    # 'max_len':np.array([100, 280]),
    # 'l1_filters':np.array([16, 32, 100, 256]),
    # 'dropout':np.array([0.50, 0.75]),
    # 'pretrained_embed':[True, False]},
    {'voc_size': [10000],
    'top': [True, False],
    'output_dim':np.array([128, 512]), 
    'max_len':np.array([100, 280]),
    'l1_filters':np.array([16, 32, 100]),
    'l2_filters':np.array([16, 32]),
    'dropout':np.array([0.50, 0.75]),
    'pretrained_embed':[True, False]},
    ]

In [0]:
model_outputs = run_param_grid(models, parameters, True)

  0%|          | 0/192 [00:00<?, ?it/s]

Epoch 1/10
50/50 [==============================] - 1s 11ms/step - loss: 1.6999 - accuracy: 0.4141 - val_loss: 1.3861 - val_accuracy: 0.5266
Epoch 2/10
50/50 [==============================] - 0s 9ms/step - loss: 1.4067 - accuracy: 0.5143 - val_loss: 1.3058 - val_accuracy: 0.5675
Epoch 3/10
50/50 [==============================] - 0s 9ms/step - loss: 1.3224 - accuracy: 0.5452 - val_loss: 1.2632 - val_accuracy: 0.5808
Epoch 4/10
50/50 [==============================] - 0s 9ms/step - loss: 1.2656 - accuracy: 0.5634 - val_loss: 1.2234 - val_accuracy: 0.5995
Epoch 5/10
50/50 [==============================] - 0s 9ms/step - loss: 1.2110 - accuracy: 0.5844 - val_loss: 1.1723 - val_accuracy: 0.6048
Epoch 6/10
50/50 [==============================] - 0s 9ms/step - loss: 1.1653 - accuracy: 0.5956 - val_loss: 1.1428 - val_accuracy: 0.6297
Epoch 7/10
50/50 [==============================] - 0s 9ms/step - loss: 1.1318 - accuracy: 0.6086 - val_loss: 1.1308 - val_accuracy: 0.6368
Epoch 8/10
50/50 [=

  1%|          | 1/192 [00:17<54:14, 17.04s/it]

Epoch 1/10
60/60 [==============================] - 1s 13ms/step - loss: 2.6499 - accuracy: 0.3397 - val_loss: 2.1489 - val_accuracy: 0.4214
Epoch 2/10
60/60 [==============================] - 1s 9ms/step - loss: 2.1599 - accuracy: 0.4458 - val_loss: 2.1396 - val_accuracy: 0.4050
Epoch 3/10
60/60 [==============================] - 1s 9ms/step - loss: 2.0523 - accuracy: 0.4661 - val_loss: 1.9967 - val_accuracy: 0.4815
Epoch 4/10
60/60 [==============================] - 1s 9ms/step - loss: 1.9773 - accuracy: 0.4766 - val_loss: 1.9827 - val_accuracy: 0.4911
Epoch 5/10
60/60 [==============================] - 1s 9ms/step - loss: 1.9236 - accuracy: 0.4916 - val_loss: 1.9517 - val_accuracy: 0.4889
Epoch 6/10
60/60 [==============================] - 1s 9ms/step - loss: 1.8831 - accuracy: 0.5054 - val_loss: 1.8988 - val_accuracy: 0.5082
Epoch 7/10
60/60 [==============================] - 1s 9ms/step - loss: 1.8422 - accuracy: 0.5101 - val_loss: 1.9256 - val_accuracy: 0.4918
Epoch 8/10
60/60 [=

  1%|          | 2/192 [00:35<54:53, 17.33s/it]

...

 99%|█████████▉| 190/192 [1:21:39<00:59, 29.51s/it]

Epoch 1/10
50/50 [==============================] - 7s 146ms/step - loss: 1.3593 - accuracy: 0.5391 - val_loss: 1.1378 - val_accuracy: 0.5968
Epoch 2/10
50/50 [==============================] - 7s 143ms/step - loss: 0.9493 - accuracy: 0.6854 - val_loss: 0.8699 - val_accuracy: 0.7300
Epoch 3/10
50/50 [==============================] - 7s 143ms/step - loss: 0.7832 - accuracy: 0.7489 - val_loss: 0.7901 - val_accuracy: 0.7629
Epoch 4/10
50/50 [==============================] - 7s 142ms/step - loss: 0.6406 - accuracy: 0.7891 - val_loss: 0.7423 - val_accuracy: 0.7824
Epoch 5/10
50/50 [==============================] - 7s 143ms/step - loss: 0.5115 - accuracy: 0.8424 - val_loss: 0.7412 - val_accuracy: 0.8206
Epoch 6/10
50/50 [==============================] - 7s 145ms/step - loss: 0.3967 - accuracy: 0.8862 - val_loss: 0.7152 - val_accuracy: 0.8321
Epoch 7/10
50/50 [==============================] - 7s 146ms/step - loss: 0.3216 - accuracy: 0.9070 - val_loss: 0.7278 - val_accuracy: 0.8401
Epoch 

 99%|█████████▉| 191/192 [1:22:54<00:43, 43.35s/it]

Epoch 1/10
60/60 [==============================] - 9s 145ms/step - loss: 2.1339 - accuracy: 0.4428 - val_loss: 1.7381 - val_accuracy: 0.5282
Epoch 2/10
60/60 [==============================] - 8s 141ms/step - loss: 1.6586 - accuracy: 0.5692 - val_loss: 1.6364 - val_accuracy: 0.5875
Epoch 3/10
60/60 [==============================] - 9s 143ms/step - loss: 1.4337 - accuracy: 0.6314 - val_loss: 1.5395 - val_accuracy: 0.6328
Epoch 4/10
60/60 [==============================] - 9s 142ms/step - loss: 1.2527 - accuracy: 0.6774 - val_loss: 1.4163 - val_accuracy: 0.6766
Epoch 5/10
60/60 [==============================] - 8s 142ms/step - loss: 1.0944 - accuracy: 0.7152 - val_loss: 1.3584 - val_accuracy: 0.6958
Epoch 6/10
60/60 [==============================] - 9s 142ms/step - loss: 0.9579 - accuracy: 0.7465 - val_loss: 1.3877 - val_accuracy: 0.7055
Epoch 7/10
60/60 [==============================] - 9s 142ms/step - loss: 0.8498 - accuracy: 0.7746 - val_loss: 1.3746 - val_accuracy: 0.7181
Epoch 

100%|██████████| 192/192 [1:24:23<00:00, 26.37s/it]


In [0]:
df_model_outputs = pd.DataFrame(model_outputs).round(3).sort_values('test_accuracy', ascending = False)
df_model_outputs.to_csv(f'{base_dir}cnn_2_model_output.csv')

In [0]:

df_model_outputs = pd.DataFrame()
model_output_csvs = []
for file in os.listdir(base_dir):
    if 'embed_model' in file:
        pass
    elif 'output.csv' in file:
        df_file = pd.read_csv(base_dir + file)
        df_file['model_name'] = file.split('.csv')[0]
        df_model_outputs = pd.concat([df_model_outputs, df_file])
df_model_outputs.drop([df_model_outputs.columns[0]], axis=1, inplace=True)
df_model_outputs.to_csv(base_dir + 'model_outputs.csv', index = False)

In [0]:
df_model_outputs.sort_values('test_accuracy', ascending = False)[:10]

,model,structure,num_parameters,accuracy,val_accuracy,test_accuracy,loss,val_loss,test_loss,elapsed_time,model_name
0,model_1,"{'dropout': 0.5, 'l1_filters': 100, 'l2_filters': 32, 'max_len': 280, 'output_dim': 512, 'pretra...",5401005,0.965,0.849,0.837,0.131,0.781,0.871,73.452,cnn_2_model_output
0,model_1,"{'dropout': 0.75, 'l1_filters': 256, 'max_len': 280, 'output_dim': 512, 'pretrained_embed': Fals...",6153993,0.964,0.852,0.835,0.120,0.789,0.932,94.808,cnn_1_model_output
1,model_1,"{'dropout': 0.5, 'l1_filters': 256, 'max_len': 280, 'output_dim': 512, 'pretrained_embed': False...",6153993,0.966,0.846,0.829,0.107,0.787,0.907,91.521,cnn_1_model_output
2,model_1,"{'dropout': 0.5, 'l1_filters': 256, 'max_len': 280, 'output_dim': 128, 'pretrained_embed': False...",2019081,0.967,0.837,0.829,0.110,0.830,0.911,31.490,cnn_1_model_output
447,model_2,"{'dropout': 0.2, 'l1_units': 128, 'l2_units': 64, 'l3_units': 64, 'max_len': 280, 'output_dim': ...",23483209,0.968,0.829,0.828,0.106,1.066,1.126,34.699,dnn_model_output
1,model_1,"{'dropout': 0.75, 'l1_filters': 100, 'l2_filters': 16, 'max_len': 280, 'output_dim': 512, 'pretr...",5388557,0.917,0.843,0.827,0.278,0.708,0.747,74.960,cnn_2_model_output
3,model_1,"{'dropout': 0.75, 'l1_filters': 32, 'max_len': 280, 'output_dim': 512, 'pretrained_embed': False...",5249257,0.957,0.850,0.827,0.158,0.741,0.835,35.160,cnn_1_model_output
5,model_1,"{'dropout': 0.5, 'l1_filters': 100, 'max_len': 280, 'output_dim': 512, 'pretrained_embed': False...",5523909,0.966,0.853,0.826,0.109,0.781,0.926,45.672,cnn_1_model_output
4,model_1,"{'dropout': 0.75, 'l1_filters': 16, 'max_len': 280, 'output_dim': 128, 'pretrained_embed': False...",1326201,0.909,0.834,0.826,0.319,0.679,0.719,10.790,cnn_1_model_output
2,model_1,"{'dropout': 0.5, 'l1_filters': 32, 'l2_filters': 32, 'max_len': 280, 'output_dim': 512, 'pretrai...",5220329,0.960,0.848,0.826,0.142,0.755,0.826,43.590,cnn_2_model_output


In [0]:
embed_results = pd.read_csv(base_dir + 'embed_model_output.csv')

In [0]:
embed_results.sort_values('test_accuracy', ascending = False)[:10]

,Unnamed: 0,model,structure,num_parameters,accuracy,val_accuracy,test_accuracy,loss,val_loss,test_loss,elapsed_time
119,98,model_1,"{'max_len': 2376, 'output_dim': 64, 'top': True, 'voc_size': 50000}",4568585,0.967,0.845,0.836,0.085,0.695,0.745,115.597
118,97,model_1,"{'max_len': 2376, 'output_dim': 64, 'top': True, 'voc_size': 10000}",2008585,0.965,0.853,0.832,0.092,0.679,0.745,105.797
117,74,model_1,"{'max_len': 280, 'output_dim': 128, 'top': True, 'voc_size': 50000}",6722569,0.965,0.837,0.832,0.086,0.721,0.779,54.971
116,104,model_1,"{'max_len': 2376, 'output_dim': 128, 'top': True, 'voc_size': 50000}",9137161,0.967,0.836,0.832,0.102,0.748,0.800,216.493
115,68,model_1,"{'max_len': 280, 'output_dim': 64, 'top': True, 'voc_size': 50000}",3361289,0.965,0.845,0.829,0.077,0.684,0.760,28.161
114,67,model_1,"{'max_len': 280, 'output_dim': 64, 'top': True, 'voc_size': 10000}",801289,0.966,0.844,0.829,0.079,0.669,0.754,21.007
113,108,model_1,"{'max_len': 2376, 'output_dim': 256, 'top': True, 'voc_size': 1000}",5730313,0.966,0.845,0.829,0.137,0.741,0.821,397.635
112,73,model_1,"{'max_len': 280, 'output_dim': 128, 'top': True, 'voc_size': 10000}",1602569,0.966,0.840,0.827,0.086,0.718,0.792,37.422
111,103,model_1,"{'max_len': 2376, 'output_dim': 128, 'top': True, 'voc_size': 10000}",4017161,0.966,0.838,0.827,0.102,0.727,0.799,197.925
110,114,model_1,"{'max_len': 2376, 'output_dim': 512, 'top': True, 'voc_size': 1000}",11460617,0.963,0.836,0.825,0.170,0.794,0.881,766.970
